In [ ]:
import praw
import pandas as pd

# ✅ Your credentials (don't share this file)
reddit = praw.Reddit(
    client_id="YOUR_CLIENT_ID",               # from https://www.reddit.com/prefs/apps
    client_secret="YOUR_CLIENT_SECRET",       # from same place
    username="YOUR_REDDIT_USERNAME",          # your Reddit username
    password="YOUR_REDDIT_PASSWORD",          # your Reddit password
    user_agent="Scraper 1.0 by/u/YOUR_USERNAME"
)

# ✅ Confirm login
print("Logged in as:", reddit.user.me())  # Should print your Reddit username

# ✅ Choose your subreddit
subreddit = reddit.subreddit("SkincareAddiction")  # or any skin care query

# ✅ Fetch posts (top 50 from 'hot' section)
posts = []
for submission in subreddit.hot(limit=50):
    posts.append({
        'title': submission.title,
        'body': submission.selftext,
        'author': str(submission.author),
        'score': submission.score,
        'num_comments': submission.num_comments,
        'created_utc': submission.created_utc,
        'permalink': f"https://reddit.com{submission.permalink}"
    })

# ✅ Convert to DataFrame
df = pd.DataFrame(posts)
print("📊 Total posts fetched:", len(df))
df.head()


Logged in as: shoban_
📊 Total posts fetched: 50


,title,body,author,score,num_comments,created_utc,permalink
0,NEW OR NEED HELP? Ask here! - ScA Daily Help T...,If you're new to SkincareAddiction: welcome!\n...,AutoModerator,1,3,1.754332e+09,https://reddit.com/r/SkincareAddiction/comment...
1,[Misc] Holy Grail Products & Ingredients - Wee...,"#Hi and welcome to the [HG Hand, Foot, & Nail ...",AutoModerator,6,3,1.743775e+09,https://reddit.com/r/SkincareAddiction/comment...
2,[Product Question] What’s your opinion on Vani...,My dermatologist recommends it. I’d be interes...,90s-modem-noise,301,246,1.754306e+09,https://reddit.com/r/SkincareAddiction/comment...
3,"I really need help, i don’t know what to do an...",So i’ve had these reoccurring blackheads on my...,AegisHS,28,46,1.754348e+09,https://reddit.com/r/SkincareAddiction/comment...
4,[Acne] pollution closed comedones?,Started a new job in central london a month ag...,Capital_Magazine_705,3,3,1.754376e+09,https://reddit.com/r/SkincareAddiction/comment...


In [29]:
# Define skin type keywords
keywords = ['oily', 'dry', 'combination', 'acne-prone', 'sensitive', 'normal']

# Function to detect mentioned skin types
def detect_skin_type(text):
    if not isinstance(text, str):
        return None
    matches = [kw for kw in keywords if kw in text.lower()]
    return ', '.join(matches) if matches else None

# Add a new column with detected skin types
df['mentioned_skin_type'] = df['body'].apply(detect_skin_type)

# Filter only posts that mention skin types
df_filtered = df[df['mentioned_skin_type'].notnull()]

print("✅ Posts mentioning skin type:", len(df_filtered))
df_filtered.head()


✅ Posts mentioning skin type: 18


,title,body,author,score,num_comments,created_utc,permalink,mentioned_skin_type
0,NEW OR NEED HELP? Ask here! - ScA Daily Help T...,If you're new to SkincareAddiction: welcome!\n...,AutoModerator,1,3,1.754332e+09,https://reddit.com/r/SkincareAddiction/comment...,"oily, dry"
10,Skin help needed please. I’m at my wits end! [...,"Hi all, I’m hoping you wonderful and incredibl...",helpmeplease2010,11,9,1.754344e+09,https://reddit.com/r/SkincareAddiction/comment...,normal
11,[Acne] what kind of acne is this?,"i never had bad acne as a teenager (im 22f), t...",CertainRespond3211,4,3,1.754358e+09,https://reddit.com/r/SkincareAddiction/comment...,dry
14,Desperately need advice. [Acne],First picture is today after 5 weeks of routin...,BigBoysenberry3474,8,4,1.754347e+09,https://reddit.com/r/SkincareAddiction/comment...,"oily, dry"
15,[Acne] I don't know what to do about these dots,for about 2 weeks now I've started to notice t...,No_Kitchen_4013,2,2,1.754369e+09,https://reddit.com/r/SkincareAddiction/comment...,oily


In [ ]:
# Save filtered results to CSV
df_filtered.to_csv("filtered_skin_posts.csv", index=False)
print("💾 Saved as 'filtered_skin_posts.csv'")



In [30]:
# Save filtered results to CSV
df_filtered.to_csv("filtered_skin_posts.csv", index=False)
print("💾 Saved as 'filtered_skin_posts.csv'")



💾 Saved as 'filtered_skin_posts.csv'


In [35]:
# Product nicknames, abbreviations, common mentions
product_aliases = {
    "niacinamide": "The Ordinary Niacinamide 10% + Zinc 1%",
    "the ordinary niacinamide": "The Ordinary Niacinamide 10% + Zinc 1%",
    "azelaic acid": "Azelaic Acid 20%",
    "cerave sa cleanser": "CeraVe Renewing SA Cleanser",
    "cerave foaming facial cleanser": "CeraVe Foaming Facial Cleanser",
    "cerave moisturizing lotion": "CeraVe Daily Moisturizing Lotion",
    "vanicream cleanser": "Vanicream Gentle Facial Cleanser",
    "vanicream lite lotion": "Vanicream Lite Moisturizing Lotion",
    "la roche posay": "La Roche-Posay Anthelios SPF 50",
    "cicaplast baume b5": "La Roche-Posay Cicaplast Baume B5",
    "adapalene": "Differin Gel (Adapalene 0.1%)",
    "biore sunscreen": "Biore UV Aqua Rich Watery Essence SPF 50+",
    "peter thomas roth": "Peter Thomas Roth Glycolic Retinol Serum",
    "murad resurfacing cleanser": "Murad AHA/BHA Exfoliating Cleanser",
    "aquaphor": "Aquaphor Healing Ointment"
}


In [36]:
def extract_product(text):
    if not isinstance(text, str):
        return None
    text_lower = text.lower()
    found = []
    for alias in product_aliases:
        if alias in text_lower:
            found.append(product_aliases[alias])
    return ", ".join(set(found)) if found else None


In [37]:
df_filtered.loc[:, 'mentioned_products'] = df_filtered['body'].apply(extract_product)


In [38]:
df_products = df_filtered[df_filtered['mentioned_products'].notnull()]
print("✅ Reviews with detected products:", len(df_products))

# Display some results
df_products[['title', 'mentioned_products', 'body']].head(5)


✅ Reviews with detected products: 11


,title,mentioned_products,body
10,Skin help needed please. I’m at my wits end! [...,"Aquaphor Healing Ointment, Murad AHA/BHA Exfol...","Hi all, I’m hoping you wonderful and incredibl..."
11,[Acne] what kind of acne is this?,The Ordinary Niacinamide 10% + Zinc 1%,"i never had bad acne as a teenager (im 22f), t..."
14,Desperately need advice. [Acne],La Roche-Posay Anthelios SPF 50,First picture is today after 5 weeks of routin...
16,how can i target this blackhead/ingrown situat...,"Aquaphor Healing Ointment, CeraVe Daily Moistu...","So I have fairly oily skin, pustules happen bu..."
17,[Review] Need help with current routine / skin...,"Peter Thomas Roth Glycolic Retinol Serum, Azel...",I will try to keep this short; I never had acn...


In [39]:
def guess_category(product_name):
    product_name = product_name.lower()
    if "cleanser" in product_name:
        return "Cleanser"
    elif "moisturiz" in product_name or "lotion" in product_name:
        return "Moisturizer"
    elif "sunscreen" in product_name or "spf" in product_name:
        return "Sunscreen"
    elif "serum" in product_name:
        return "Serum"
    elif "ointment" in product_name:
        return "Ointment"
    elif "acid" in product_name:
        return "Exfoliant / Acid"
    else:
        return "Other"


In [40]:
# Set to store unique product names
unique_products = set()

for row in df_products['mentioned_products'].dropna():
    products = [p.strip() for p in row.split(',')]
    unique_products.update(products)

# Build a clean product database
product_db = []

for product in unique_products:
    brand = product.split()[0]  # assume first word = brand
    category = guess_category(product)
    
    product_db.append({
        'brand': brand,
        'product_name': product,
        'category': category
    })

df_product_db = pd.DataFrame(product_db)


In [41]:
print("🧴 Unique Products Extracted:", len(df_product_db))
df_product_db.head()


🧴 Unique Products Extracted: 12


,brand,product_name,category
0,CeraVe,CeraVe Renewing SA Cleanser,Cleanser
1,CeraVe,CeraVe Daily Moisturizing Lotion,Moisturizer
2,Differin,Differin Gel (Adapalene 0.1%),Other
3,The,The Ordinary Niacinamide 10% + Zinc 1%,Other
4,Azelaic,Azelaic Acid 20%,Exfoliant / Acid


In [43]:
df_products.to_csv("reviews_with_products.csv", index=False)
df_product_db.to_csv("product_database.csv", index=False)


In [44]:
import re

# Skin types
skin_type_keywords = ["oily", "dry", "combination", "sensitive", "normal", "acne-prone", "dehydrated"]

# Common locations (expand as needed)
location_keywords = ["USA", "India", "Mexico", "Canada", "UK", "Philippines"]

# Simple pattern to find age like "I'm 22", "22F", etc.
age_pattern = re.compile(r'\b(\d{2})(?:\s?f|\s?m)?\b', re.IGNORECASE)


In [45]:
def detect_skin_type(text):
    if not isinstance(text, str):
        return None
    text = text.lower()
    for kw in skin_type_keywords:
        if kw in text:
            return kw
    return None

def detect_location(text):
    if not isinstance(text, str):
        return None
    for loc in location_keywords:
        if loc.lower() in text.lower():
            return loc
    return None

def detect_age(text):
    if not isinstance(text, str):
        return None
    match = age_pattern.search(text)
    if match:
        return int(match.group(1))
    return None


In [49]:
df_products.loc[:, 'detected_skin_type'] = df_products['body'].apply(detect_skin_type)
df_products.loc[:, 'detected_age'] = df_products['body'].apply(detect_age)
df_products.loc[:, 'detected_location'] = df_products['body'].apply(detect_location)


In [48]:
df_products = df_filtered[df_filtered['mentioned_products'].notnull()].copy()


In [50]:
df_products[['title', 'detected_skin_type', 'detected_age', 'detected_location']].head(10)


,title,detected_skin_type,detected_age,detected_location
10,Skin help needed please. I’m at my wits end! [...,normal,NaN,Mexico
11,[Acne] what kind of acne is this?,dry,22.0,None
14,Desperately need advice. [Acne],oily,16.0,None
16,how can i target this blackhead/ingrown situat...,oily,NaN,None
17,[Review] Need help with current routine / skin...,oily,10.0,UK
18,[Routine Help] Flaky white patches around mout...,dry,NaN,None
29,Stuck in an endless cycle of dryness [Routine ...,dry,NaN,None
31,[Acne] Need advice,oily,25.0,None
34,[Routine Help] 16 year old trying to figure ou...,combination,NaN,Canada
42,How to know if skin tolerates Adapalene? [Rout...,dry,NaN,None


In [52]:
df_product_db = df_product_db.reset_index(drop=True)
df_product_db['product_id'] = df_product_db.index + 1  # ID starts from 1


In [53]:
# Left join to get product_id from product_db
df_reviews = df_products.merge(
    df_product_db[['product_name', 'product_id']],
    left_on='mentioned_products',
    right_on='product_name',
    how='left'
)


In [54]:
products_table = df_product_db[['product_id', 'brand', 'product_name', 'category']]


In [57]:
df_reviews.columns.tolist()


['title',
 'body',
 'author',
 'score',
 'num_comments',
 'created_utc',
 'permalink',
 'mentioned_skin_type',
 'mentioned_products',
 'detected_skin_type',
 'detected_age',
 'detected_location',
 'product_name',
 'product_id']

In [58]:
reviews_table = df_reviews[[
    'product_id',
    'title',
    'body',
    'author',
    'created_utc',
    'permalink',
    'detected_skin_type',
    'detected_age',
    'detected_location'
]].rename(columns={
    'author': 'username',
    'body': 'review_content',
    'created_utc': 'review_date',
    'permalink': 'url'
})


In [61]:
reviews_table.head(100)



,product_id,title,review_content,username,review_date,url,detected_skin_type,detected_age,detected_location
0,NaN,Skin help needed please. I’m at my wits end! [...,"Hi all, I’m hoping you wonderful and incredibl...",helpmeplease2010,1.754344e+09,https://reddit.com/r/SkincareAddiction/comment...,normal,NaN,Mexico
1,4.0,[Acne] what kind of acne is this?,"i never had bad acne as a teenager (im 22f), t...",CertainRespond3211,1.754358e+09,https://reddit.com/r/SkincareAddiction/comment...,dry,22.0,None
2,12.0,Desperately need advice. [Acne],First picture is today after 5 weeks of routin...,BigBoysenberry3474,1.754347e+09,https://reddit.com/r/SkincareAddiction/comment...,oily,16.0,None
3,NaN,how can i target this blackhead/ingrown situat...,"So I have fairly oily skin, pustules happen bu...",DevotedDissent,1.754355e+09,https://reddit.com/r/SkincareAddiction/comment...,oily,NaN,None
4,NaN,[Review] Need help with current routine / skin...,I will try to keep this short; I never had acn...,naratcis,1.754380e+09,https://reddit.com/r/SkincareAddiction/comment...,oily,10.0,UK
5,8.0,[Routine Help] Flaky white patches around mout...,"Hi everyone, I’m dealing with a confusing skin...",CityMost9918,1.754380e+09,https://reddit.com/r/SkincareAddiction/comment...,dry,NaN,None
6,NaN,Stuck in an endless cycle of dryness [Routine ...,Curious if anyone else has had a similar exper...,Spec2014,1.754363e+09,https://reddit.com/r/SkincareAddiction/comment...,dry,NaN,None
7,4.0,[Acne] Need advice,I’m a 25-year-old female and I’ve been living ...,Mysterious_Spell_732,1.754374e+09,https://reddit.com/r/SkincareAddiction/comment...,oily,25.0,None
8,NaN,[Routine Help] 16 year old trying to figure ou...,I have combination skin and the problems im ha...,SpecialPressure2031,1.754374e+09,https://reddit.com/r/SkincareAddiction/comment...,combination,NaN,Canada
9,3.0,How to know if skin tolerates Adapalene? [Rout...,"I’ve started using adapalene 0.1%, I’ve only u...",thoughtmachine13,1.754371e+09,https://reddit.com/r/SkincareAddiction/comment...,dry,NaN,None


In [63]:
reviews_table.to_csv("reviews_table.csv", index=False)


In [64]:
# Save reviews table
reviews_table.to_csv("reviews_table.csv", index=False)

# Save products table
products_table.to_csv("products_table.csv", index=False)
